In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
from os.path import join, isfile
from os import listdir
from functools import partial

import glam
import parameter_recovery as pr

%load_ext autoreload
%autoreload 2

/Users/felixmolter/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Systematic parameter recovery (NUTS)

Here, we perform a structured and systematic parameter recovery study. We use the original model variant with 4 parameters ($v$, $\gamma$, $\sigma$, $\tau$). For each parameter, we define a range of sensible values and select a `low`, `medium` and `high` value, based on the individual parameter estimates we obtained by hierarchically fitting the GLAM to the data from Krajbich & Rangel (2011) in Thomas, Molter, Krajbich, Heekeren & Mohr (submitted).

We then pick one parameter, one constellation of values of the remaining three parameters (e.g., we pick $v$, and set $\gamma$, $\sigma$ and $\tau$ to their `low` values). Then, for 10 different values of $v$ along its range, we generate a synthetic dataset for a single participant with N=100 trials, random item values and gaze, fit the model and record the generating and estimated parameters. The procedure is repeated for all possible constellation of other parameter values, and other parameters, resulting in 1080 ($10 \times 4 \times 3^3$) simulated and fitted datasets total.

In [2]:
output_folder = join('results', 'parameter-recovery_individual', 'nuts')
label = 'individual_nuts'

## Define parameter ranges from 2011 estimates

In [3]:
bounds = dict(v=[0.000015, 0.00015],
              gamma=[-1, 1],
              s=[0.004, 0.011],
              tau=[0.1, 1.25])

ranges = {parameter: np.linspace(*bounds[parameter], 10)
          for parameter in ['v', 'gamma', 's', 'tau']}
ranges

{'v': array([1.50e-05, 3.00e-05, 4.50e-05, 6.00e-05, 7.50e-05, 9.00e-05,
        1.05e-04, 1.20e-04, 1.35e-04, 1.50e-04]),
 'gamma': array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
         0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ]),
 's': array([0.004     , 0.00477778, 0.00555556, 0.00633333, 0.00711111,
        0.00788889, 0.00866667, 0.00944444, 0.01022222, 0.011     ]),
 'tau': array([0.1       , 0.22777778, 0.35555556, 0.48333333, 0.61111111,
        0.73888889, 0.86666667, 0.99444444, 1.12222222, 1.25      ])}

In [4]:
# Determine low, medium, high values as 10th, 50th and 90th percentile of these ranges
values = {parameter: np.percentile(np.linspace(*bounds[parameter], 100), [10, 50, 90])
          for parameter in ['v', 'gamma', 's', 'tau']}
values

{'v': array([2.850e-05, 8.250e-05, 1.365e-04]),
 'gamma': array([-8.00000000e-01,  5.55111512e-17,  8.00000000e-01]),
 's': array([0.0047, 0.0075, 0.0103]),
 'tau': array([0.215, 0.675, 1.135])}

In [5]:
parameter_info = {parameter: dict(low=values[parameter][0],
                                  medium=values[parameter][1],
                                  high=values[parameter][2],
                                  variable=ranges[parameter])
                  for parameter in ['v', 'gamma', 's', 'tau']}

## Run the recovery in parallel

In [6]:
partial_output_folder = join(output_folder, 'partial')

simulate_group_args = dict(kind='individual',
                           n_individuals=1,
                           n_trials=100, 
                           n_items=3,
                           error_weight=0,
                           value_range=[1, 10],
                           seed=None)

make_model_args = dict(kind='individual',
                       t0_val=0,
                       error_weight=0)

fit_args = dict(method='MCMC',
                draws=10000,
                step=pm.NUTS,
                tune=2500,
                cores=1,
                progressbar=False)

wrapper = partial(pr.recover_and_save,
                  output_folder=partial_output_folder,
                  label=label,
                  simulate_group_args=simulate_group_args,
                  make_model_args=make_model_args,
                  fit_args=fit_args)

In [ ]:
from multiprocessing import Pool

n_cores = 14
p = Pool(n_cores)

output = p.map(wrapper, pr.generate_parameter_sets(parameter_info))

Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Generating single subject models for 1 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4588')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4592')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4599')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4600')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4598')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4596')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/c

INFO (theano.gof.compilelock): Waiting for existing lock by process '4510' (I am process '4590')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4510' (I am process '4593')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4510' (I am process '4589')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4510' (I am process '4597')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4510' (I am process '4587')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): T

INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4596')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4600')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4598')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4599')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i3

INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4597')
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4594')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '4595' (I am process '4591')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/felixmolter/.theano/compiledir_Darwin-17.7.0-x86_64-i386-64bit-i386-3.6.8-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, dele

...

Sequential sampling (2 chains in 1 job)
NUTS: [tau, s, gamma, v]
The acceptance probability does not match the target. It is 0.5825779823738396, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The acceptance probability does not match the target. It is 0.6424633710031756, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The gelman-rubin statistic is larger than 1.2 for some parameters.
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...
Generating single subject models for 1 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


Sequential sampling (2 chains in 1 job)
NUTS: [tau, s, gamma, v]
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


/!\ Automatically setting parameter precision...
Generating single subject models for 1 subjects...
Fitting 1 model(s) using MCMC...
  Fitting model 1 of 1...


Sequential sampling (2 chains in 1 job)
NUTS: [tau, s, gamma, v]


## Combine results

In [ ]:
filenames = [file for file in listdir(partial_output_folder)
             if file.endswith('.csv')]

partial_recoveries = []

for file in filenames:
    partial = pd.read_csv(join(partial_output_folder, file))
    partial_recoveries.append(partial)

parameter_recovery = pd.concat(partial_recoveries).sort_values('index').reset_index(drop=True)
del partial_recoveries
parameter_recovery.to_csv(join(output_folder, 'parameter-recovery_', label, '.csv'))
parameter_recovery.head()